### 오디오 다운로드

In [ ]:
from predict_stock_outlook import extract_video_audio

df = pd.read_csv("data_kr/video/자료 수집 최종본.csv")

for row in df.itertuples():
    if pd.isna(row.url) or row.url == '' or row.category != "video":
        continue

    code = str(row.code).zfill(6)
    audio_dir = f'data_kr/video/audio/{row.sector}/{code}/'
    text_dir = f'data_kr/video/text/{row.sector}/{code}/'
    os.makedirs(audio_dir, exist_ok=True)

    if audio_dir + f'{row.year}-{row.quarter}' != 'data_kr/video/audio/산업재/003490/2016-Q2':
        continue
    
    if extract_video_audio("link", row.url, audio_dir + f'{row.year}-{row.quarter}'):
        with open('data_kr/video/log.txt', "a", encoding="utf-8") as log_file:
            timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
            log_file.write(f"{timestamp} audio download completed: {audio_dir + f'{row.year}-{row.quarter}'}\n")
    else:
        with open('data_kr/video/log.txt', "a", encoding="utf-8") as log_file:
            timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
            log_file.write(f"{timestamp} audio download error: {audio_dir + f'{row.year}-{row.quarter}'}\n")

### 텍스트로 변환

In [ ]:
from predict_stock_outlook import audio2text

df = pd.read_csv("data_kr/video/자료 수집 최종본.csv")

for row in df.itertuples():
    if pd.isna(row.url) or row.url == '' or row.category != "video":
        continue

    code = str(row.code).zfill(6)
    audio_dir = f'data_kr/video/audio/{row.sector}/{code}/'
    text_dir = f'data_kr/video/text/{row.sector}/{code}/'
    os.makedirs(text_dir, exist_ok=True)
    
    if audio_dir + f'{row.year}-{row.quarter}' != 'data_kr/video/audio/산업재/003490/2016-Q2':
        continue
    
    try:
        text = audio2text(audio_dir + f'{row.year}-{row.quarter}')
        with open(text_dir + f'{row.year}-{row.quarter}.txt', "w", encoding="utf-8") as f:
            f.write(text)
        with open('data_kr/video/log.txt', "a", encoding="utf-8") as log_file:
            timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
            log_file.write(f"{timestamp} whisper completed: {text_dir + f'{row.year}-{row.quarter}'}\n")
    except Exception as e:
        with open('data_kr/video/log.txt', "a", encoding="utf-8") as log_file:
            timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
            log_file.write(f"{timestamp} whisper error: {text_dir + f'{row.year}-{row.quarter}'}\n")

### 텍스트 token 수 확인

In [ ]:
df = pd.read_csv("data_kr/video/자료 수집 최종본.csv")

import tiktoken
# 예: GPT-4용 인코더 불러오기
encoding = tiktoken.encoding_for_model("gpt-4o")
total_tokens = 0

for row in tqdm(df.itertuples(), total=len(df), desc="checking tokens"):
	if pd.isna(row.url) or row.url == '':
		continue

	code = str(row.code).zfill(6)
	text_dir = f'data_kr/video/text/{row.sector}/{code}/'
	os.makedirs(text_dir, exist_ok=True)
    
	try:
		filename = f'{row.year}-{row.quarter}-{str(row.month).zfill(2)}-{row.week}.txt'
		with open(text_dir + filename, "r", encoding="utf-8") as file:
			text = file.read()
			system_prompt = """
		너는 경제 전문 뉴스 분석 AI야. 사용자가 지정한 종목(회사명)과 직접적으로 관련된 정보만 선택해 핵심적으로 요약해.
		사실 기반으로 요약하고, 감성이나 추론이 필요한 경우에는 중립적으로 표현해.
		"""

			user_prompt = f"""
		다음은 경제 뉴스 기사입니다.

		이 기사에서 **한국 상장 기업 "{code}"**과 관련된 내용만 골라 요약해 주세요.

		요약 기준:
		- "{code}"이 언급된 부분 중심
		- 관련 사업, 실적, 주가, 시장 반응, 경쟁사와의 연관성
		- 정부 정책, 산업 트렌드 등 외부 요인 중 관련 있는 부분
		- 부정적/긍정적 논조도 간단히 언급 (있는 경우)

		형식은 간결한 문장 또는 Bullet Point 형식으로 작성해 주세요.

		기사 전문:
		{text}
		"""
			total_prompt = system_prompt + user_prompt
			tokens = encoding.encode(total_prompt)
			total_tokens += len(tokens)
		with open('data_kr/video/num_token.txt', "a", encoding="utf-8") as log_file:
			timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
			log_file.write(f"{text_dir}{filename} len: {len(text)}, token: {len(tokens)}\n")
	except Exception as e:
		with open('data_kr/video/num_token.txt', "a", encoding="utf-8") as log_file:
			timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
			log_file.write(f"{text_dir}{filename} error: " + e + "\n")

with open('data_kr/video/num_token.txt', "a", encoding="utf-8") as log_file:
	log_file.write(f"total tokens: {total_tokens}\n")

### LLM으로 요약

In [ ]:
from predict_stock_outlook import summarize_text

df = pd.read_csv("data_kr/video/자료 수집 최종본.csv")

for row in tqdm(df.itertuples(), total=len(df), desc="LLM summarizing"):
    if pd.isna(row.url) or row.url == '':
        continue

    code = str(row.code).zfill(6)
    name = row.name
    text_dir = f'data_kr/video/text/{row.sector}/{code}/'
    summary_dir = f'preprocessed_data/llm/summary/{row.sector}/{code}/'
    os.makedirs(summary_dir, exist_ok=True)
        
    try:
        filename = f'{row.year}-{row.quarter}-{str(row.month).zfill(2)}-{row.week}.txt'
        stock = f'{name}({code})'
        with open(text_dir + filename, "r", encoding="utf-8") as file:
            text = file.read()
        summary = summarize_text(text, stock)
        with open(summary_dir + filename, "w", encoding="utf-8") as f:
            f.write(summary)
        with open('preprocessed_data/llm/summary/log.txt', "a", encoding="utf-8") as log_file:
            timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
            log_file.write(f"{timestamp} summary completed: {summary_dir + filename}\n")
    except Exception as e:
        with open('preprocessed_data/llm/summary/log.txt', "a", encoding="utf-8") as log_file:
            timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
            log_file.write(f"{timestamp} summary error: {summary_dir + filename}\t error: {e}\n")

### LLM으로 기사 자막요약을 통해 등락 예측

In [37]:
from predict_stock_outlook import predict_market_from_summary
import os
from tqdm import tqdm
from datetime import datetime
import pandas as pd

df = pd.read_csv('data_kr/video/뉴스 기사 수집본.csv', encoding='utf-8')

for code in df["code"].unique():
    df_ = df[df["code"] == code].reset_index(drop=True)
    
    for row in tqdm(df_.itertuples(), total=len(df_), desc=f"{code}LLM predicting"):
        if pd.isna(row.url) or row.url == '':
            continue
        
        code = str(row.code).zfill(6)
        name = row.name
        article_dir = f'data_kr/video/text/{row.sector}/{code}/'
        predict_dir = f'preprocessed_data/llm/predict_text/{row.sector}/{code}/'
        os.makedirs(predict_dir, exist_ok=True)

        try:
            filename = f'{row.year}-{row.quarter}-{str(row.month).zfill(2)}-{row.week}.txt'
            stock = f'{name}({code})'
            with open(f'{article_dir}{filename}', "r", encoding="utf-8") as file:
                article = file.read()
            data = predict_market_from_summary(article, f'{name}({code})')
            
            with open(f'{predict_dir}{filename}', "w", encoding="utf-8") as file:
                file.write(data)
            
            with open('preprocessed_data/llm/predict_text/log.txt', "a", encoding="utf-8") as log_file:
                timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
                log_file.write(f"{timestamp} predict completed: {predict_dir}{filename}\n")
        except Exception as e:
            with open('preprocessed_data/llm/predict_text/log.txt', "a", encoding="utf-8") as log_file:
                timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
                log_file.write(f"{timestamp} predict error: {predict_dir}{filename}\n")
                print(e)

86280LLM predicting: 100%|██████████████████████████████████████████████████| 204/204 [00:00<00:00, 11823.27it/s]

3317


### LLM으로 영상 자막요약을 통해 등락 예측

In [2]:
from predict_stock_outlook import predict_market_from_summary
import os
from tqdm import tqdm
from datetime import datetime
import pandas as pd

df = pd.read_csv('data_kr/video/뉴스 영상 수집본.csv', encoding='utf-8')

for code in df["code"].unique():
    df_ = df[df["code"] == code].reset_index(drop=True)

    for row in tqdm(df_.itertuples(), total=len(df_), desc=f"{code}LLM predicting"):
        if pd.isna(row.url) or row.url == '':
            continue
        
        code = str(row.code).zfill(6)	
        name = row.name
        script_dir = f'data_kr/video/script/{row.sector}/{code}/'
        predict_dir = f'preprocessed_data/llm/predict_video/{row.sector}/{code}/'
        os.makedirs(predict_dir, exist_ok=True)

        try:
            filename = f'{row.year}-{row.quarter}-{str(row.month).zfill(2)}-{row.week}.txt'
            stock = f'{name}({code})'
            with open(f'{script_dir}{filename}', "r", encoding="utf-8") as file:
                script = file.read()
            prediction = predict_market_from_summary(script, f'{name}({code})')
            
            with open(f'{predict_dir}{filename}', "w", encoding="utf-8") as file:
                file.write(prediction)
            
            with open('preprocessed_data/llm/predict_video/log.txt', "a", encoding="utf-8") as log_file:
                timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
                log_file.write(f"{timestamp} predict completed: {predict_dir}{filename}\n")
        except Exception as e:
            with open('preprocessed_data/llm/predict_video/log.txt', "a", encoding="utf-8") as log_file:
                timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
                log_file.write(f"{timestamp} predict error: {predict_dir}{filename}\n")  

86280LLM predicting: 100%|█████████████████████████████████████████████████████| 204/204 [02:20<00:00,  1.45it/s]


### LLM으로 영상자막요약 + 기사자막요약을 통해 등락 예측

In [ ]:
from predict_stock_outlook import predict_market_from_mix
import os
from tqdm import tqdm
from datetime import datetime
import pandas as pd

df_v = pd.read_csv('data_kr/video/뉴스 영상 수집본.csv', encoding='utf-8')
df_v.rename(columns={'url':'v_url', 'upload_dt':'v_upload_dt'}, inplace=True)
df_v = df_v[["year","quarter","month","week","code","name","sector","after","before","v_url","v_upload_dt"]]

df_a = pd.read_csv('data_kr/video/뉴스 기사 수집본.csv', encoding='utf-8')
df_a.rename(columns={'url':'a_url', 'upload_dt':'a_upload_dt'}, inplace=True)
df_a = df_a[["a_url","a_upload_dt"]]

df = pd.concat([df_v, df_a], axis=1)

for code in df["code"].unique():
    df_ = df[df["code"] == code].reset_index(drop=True)

    for row in tqdm(df_.itertuples(), total=len(df_), desc=f"{code}LLM predicting"):
        if pd.isna(row.v_url) or pd.isna(row.a_url) :
            continue
        
        code = str(row.code).zfill(6)
        name = row.name
        v_summary_dir = f'data_kr/video/script/{row.sector}/{code}/'
        t_summary_dir = f'data_kr/video/text/{row.sector}/{code}/'
        predict_dir = f'preprocessed_data/llm/predict_mix/{row.sector}/{code}/'
        os.makedirs(predict_dir, exist_ok=True)

        try:
            filename = f'{row.year}-{row.quarter}-{str(row.month).zfill(2)}-{row.week}.txt'
            stock = f'{name}({code})'
            with open(f'{v_summary_dir}{filename}', "r", encoding="utf-8") as file:
                script = file.read()
            with open(f'{t_summary_dir}{filename}', "r", encoding="utf-8") as file:
                article = file.read()
 
            data = predict_market_from_mix(article, script, f'{name}({code})')
            
            with open(f'{predict_dir}{filename}', "w", encoding="utf-8") as file:
                file.write(data)
            
            with open('preprocessed_data/llm/predict_mix/log.txt', "a", encoding="utf-8") as log_file:
                timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
                log_file.write(f"{timestamp} predict completed: {predict_dir}{filename}\n")
        except Exception as e:
            with open('preprocessed_data/llm/predict_mix/log.txt', "a", encoding="utf-8") as log_file:
                timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
                log_file.write(f"{timestamp} predict error: {predict_dir}{filename}\n")

120LLM predicting:  80%|████████████████████████████████████████████▏          | 164/204 [04:10<01:06,  1.65s/it]

### 기사 예측 결과 정리

In [23]:
import json
import os
from tqdm import tqdm
from datetime import datetime
import pandas as pd

df=pd.read_csv('./data_kr/video/뉴스 기사 수집본.csv', encoding='utf-8')

for code in df["code"].unique():
    df_ = df[df["code"] == code].reset_index(drop=True)
    predict_list = []
    reason_list = []
    score_list = []

    for row in tqdm(df_.itertuples(), total=len(df_), desc=f"{code}LLM predicting"):
        if pd.isna(row.url) or row.url == '':
            predict_list.append(None)
            reason_list.append(None)
            score_list.append(None)
            continue
        
        code = str(row.code).zfill(6)	
        name = row.name
        predict_dir = f'preprocessed_data/llm/predict_text/{row.sector}/{code}/'
        os.makedirs(predict_dir, exist_ok=True)

        try:
            filename = f'{row.year}-{row.quarter}-{str(row.month).zfill(2)}-{row.week}.txt'
 
            with open(f'{predict_dir}{filename}', 'r', encoding='utf-8') as f:
                # json.load() 함수를 사용하여 파일의 내용을 파이썬 딕셔너리로 불러옵니다.
                data_dict = json.load(f)

            predict_list.append(data_dict["sentiment"])
            reason_list.append(data_dict["reasoning"])
            score_list.append(data_dict["score"])
            
        except Exception as e:
            predict_list.append("중립")
            reason_list.append("관련 없음")
            score_list.append(0)
            
    df_predict = df_.copy()
    df_predict["prediction"] = predict_list
    df_predict["reason"] = reason_list
    df_predict["score"] = score_list
    df_predict = df_predict[["year", "quarter", "month", "week", "code", "name", "sector", "upload_dt", "prediction", "reason", "score"]]
    df_predict.to_csv(f"{predict_dir}{code}.csv", index=False, encoding="utf-8")

86280LLM predicting: 100%|████████████████████████████████████████████████████| 204/204 [00:00<00:00, 367.16it/s]


### 영상 예측 결과 정리

In [24]:
df=pd.read_csv('./data_kr/video/뉴스 영상 수집본.csv', encoding='utf-8')
for code in df["code"].unique():
    df_ = df[df["code"] == code].reset_index(drop=True)
    predict_list = []
    reason_list = []
    score_list = []

    for row in tqdm(df_.itertuples(), total=len(df_), desc=f"{code}LLM predicting"):
        if pd.isna(row.url) or row.url == '':
            predict_list.append(None)
            reason_list.append(None)
            score_list.append(None)
            continue
        
        code = str(row.code).zfill(6)	
        name = row.name
        predict_dir = f'preprocessed_data/llm/predict_video/{row.sector}/{code}/'
        os.makedirs(predict_dir, exist_ok=True)

        try:
            filename = f'{row.year}-{row.quarter}-{str(row.month).zfill(2)}-{row.week}.txt'

            with open(f'{predict_dir}{filename}', 'r', encoding='utf-8') as f:
                # json.load() 함수를 사용하여 파일의 내용을 파이썬 딕셔너리로 불러옵니다.
                data_dict = json.load(f)

            predict_list.append(data_dict["sentiment"])
            reason_list.append(data_dict["reasoning"])
            score_list.append(data_dict["score"])
            
        except Exception as e:
            predict_list.append("불가능")
            reason_list.append("관련 없음")
            score_list.append(0)
            
    df_predict = df_.copy()
    df_predict["prediction"] = predict_list
    df_predict["reason"] = reason_list
    df_predict["score"] = score_list
    df_predict = df_predict[["year", "quarter", "month", "week", "code", "name", "sector", "upload_dt", "prediction", "reason", "score"]]
    df_predict.to_csv(f"{predict_dir}{code}.csv", index=False, encoding="utf-8")

86280LLM predicting: 100%|████████████████████████████████████████████████████| 204/204 [00:00<00:00, 491.22it/s]


### 영상+기사 예측 결과 정리

In [25]:
from predict_stock_outlook import predict_market_from_mix
import os
from tqdm import tqdm
from datetime import datetime
import pandas as pd
import numpy as np

df_v = pd.read_csv('data_kr/video/뉴스 영상 수집본.csv', encoding='utf-8')
df_v.rename(columns={'url':'v_url', 'upload_dt':'v_upload_dt'}, inplace=True)
df_v = df_v[["year","quarter","month","week","code","name","sector","after","before","v_url","v_upload_dt"]]

df_a = pd.read_csv('data_kr/video/뉴스 기사 수집본.csv', encoding='utf-8')
df_a.rename(columns={'url':'a_url', 'upload_dt':'a_upload_dt'}, inplace=True)
df_a = df_a[["a_url","a_upload_dt"]]

df = pd.concat([df_v, df_a], axis=1)

for code in df["code"].unique():
    df_ = df[df["code"] == code].reset_index(drop=True)
    predict_list = []
    reason_list = []
    score_list = []
    uploaddt_list = []

    for row in tqdm(df_.itertuples(), total=len(df_), desc=f"{code}LLM predicting"):
        if pd.isna(row.v_url) or pd.isna(row.a_url) :
            predict_list.append(None)
            reason_list.append(None)
            score_list.append(None)
            uploaddt_list.append(None)
            continue
        
        code = str(row.code).zfill(6)
        name = row.name
        predict_dir = f'preprocessed_data/llm/predict_mix/{row.sector}/{code}/'
        os.makedirs(predict_dir, exist_ok=True)

        try:
            filename = f'{row.year}-{row.quarter}-{str(row.month).zfill(2)}-{row.week}.txt'
            
            with open(f'{predict_dir}{filename}', 'r', encoding='utf-8') as f:
                # json.load() 함수를 사용하여 파일의 내용을 파이썬 딕셔너리로 불러옵니다.
                data_dict = json.load(f)

            predict_list.append(data_dict["sentiment"])
            reason_list.append(data_dict["reasoning"])
            score_list.append(data_dict["score"])
            if not pd.isna(row.v_upload_dt) and not pd.isna(row.a_upload_dt):
                uploaddt = row.v_upload_dt if row.v_upload_dt > row.a_upload_dt else row.a_upload_dt
                uploaddt_list.append(uploaddt)
            else:
                uploaddt_list.append(None)
                
            
        except Exception as e:
            print(f'error: {e}, file:{filename}')
            predict_list.append("불가능")
            reason_list.append("관련 없음")
            score_list.append(0)
            
            
    df_predict = df_.copy()
    df_predict["prediction"] = predict_list
    df_predict["reason"] = reason_list
    df_predict["score"] = score_list
    df_predict['upload_dt'] = uploaddt_list
    df_predict = df_predict[["year", "quarter", "month", "week", "code", "name", "sector", "upload_dt", "prediction", "reason", "score"]]
    df_predict.to_csv(f"{predict_dir}{code}.csv", index=False, encoding="utf-8")

86280LLM predicting: 100%|████████████████████████████████████████████████████| 204/204 [00:00<00:00, 614.10it/s]


## 예측 결과 정리 concat

In [10]:
from pathlib import Path
import pandas as pd
# 현재 폴더를 Path 객체로 만듭니다.
# datas=['video', 'text', 'mix']
datas=['text']
for data in datas:
    path_list = []
    for sector in ["산업재", "정보기술"]:
        root_path = Path(f'./preprocessed_data/llm/predict_{data}/{sector}')
        
        # rglob('*')는 재귀적으로 모든(*) 파일과 폴더를 찾습니다.
        for path in root_path.rglob('*.csv'):
            # path는 파일 또는 폴더에 대한 Path 객체입니다.
            path_list.append(path)
    df_list = [pd.read_csv(f'{path}', encoding='utf-8') for path in path_list]
    df = pd.concat(df_list, axis=0)
    df = df.sort_values(by=['code', 'year', 'quarter', 'month', 'week'], ascending=True)
    df.to_csv(f'./preprocessed_data/llm/predict_{data}/predict.csv', index=False, encoding='utf-8')
    df.to_csv(f'./preprocessed_data/llm/predict_total/predict_{data}.csv', index=False, encoding='utf-8')

## mix에 video or text 단독예측도 추가

In [11]:
import pandas as pd

df_v = pd.read_csv('preprocessed_data/llm/predict_total/predict_video.csv', encoding='utf-8')
df_t = pd.read_csv('preprocessed_data/llm/predict_total/predict_text.csv', encoding='utf-8')
df_m = pd.read_csv('preprocessed_data/llm/predict_total/predict_mix.csv', encoding='utf-8')

df_total = df_m.copy()
df_total[pd.isna(df_total)] = df_v[pd.isna(df_total)]
df_total[pd.isna(df_total)] = df_t[pd.isna(df_total)]

df_total.to_csv('preprocessed_data/llm/predict_total/predict_total.csv', index=False, encoding='utf-8')

## LLM 정확도 측정 (업로드일과 일주일 마감일 가격 비교)

In [12]:
from datetime import timedelta, datetime
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np

def get_pred_accuracy():
    plt.figure(figsize=(24, 5))
    
    datas = ['video', 'text', 'mix', 'total']
    
    for idx, data in enumerate(datas):
        df = pd.read_csv(f'preprocessed_data/llm/predict_total/predict_{data}.csv', encoding='utf-8')
        df_d = pd.read_csv(f'data_kr/video/뉴스 영상 수집본.csv', encoding='utf-8')[['after', 'before']]
        df = pd.concat([df, df_d], axis=1)
        df[['upload_dt', 'after', 'before']] = df[['upload_dt', 'after', 'before']].apply(pd.to_datetime)
        
        ### LLM 예측에 실제 등락 라벨 추가 ###
        df["code"] = df["code"].astype(str).str.zfill(6)
        
        price_upload = [] # 업로드 당일 종가
        price_end = [] # 공시 당일 종가
        for row in tqdm(df.itertuples(), total=len(df)):
            df_price = pd.read_csv(f"data_kr/price/{row.code}.csv")
            df_price['날짜'] = pd.to_datetime(df_price['날짜'])
            ### 업로드 날짜 직전 종가
            price_upload.append(df_price.loc[df_price["날짜"] < row.upload_dt, "종가"].iloc[-1] if not pd.isna(row.upload_dt) else None)
            ### 데이터 업로드 주간 마지막 날 종가
            price_end.append(df_price.loc[df_price["날짜"] < row.before, "종가"].iloc[-1] if not pd.isna(row.before) else None)
        
        df["price_upload"] = price_upload
        df["price_end"] = price_end
    
        def check_change(row):
            rate = (row["price_end"] / row["price_upload"] - 1) * 100
            if rate > 0:
                return "상승"
            else:
                return "하락"
        df["label"] = df.apply(check_change, axis=1)

        ###############################################################
        ### 유효한 데이터만 남기기 ###
        valid_bool = ~(pd.isna(df['prediction']) | (df['prediction'] == '중립'))
        prediction = df[valid_bool]['prediction']
        label = df[valid_bool]['label']
        
        mapping = {'매우 긍정': '상승', '긍정': '상승', '중립': '횡보', '부정': '하락', '매우 부정': '하락'}
        prediction = [mapping[l] for l in prediction]
        
        # 혼동 행렬
        labels = ["상승", "하락"]
        cm = confusion_matrix(label, prediction, labels=labels)
        
        # 한글 폰트 설정 (Windows, Mac, Linux 환경에 맞게 설정)
        # 윈도우
        plt.rc('font', family='Malgun Gothic')
    
        # 시각화
        plt.subplot(1, 4, idx + 1)  # 1행 3열 중 i번째 위치에 subplot을 생성
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
        plt.xlabel('Prediction')
        plt.ylabel('Label')
        plt.title(f'{data} Accuracy Confusion Matrix')
        
    plt.tight_layout()
    plt.savefig(f"preprocessed_data/llm/predict_total/predict_accuracy.png")
    plt.close()

get_pred_accuracy()

100%|███████████████████████████████████████████████████████████████████████| 5712/5712 [00:25<00:00, 225.77it/s]


## LLM 예측 정확도 지표 계산

In [23]:
from datetime import timedelta, datetime
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

def get_pred_accuracy():
    scores = dict()
    
    datas = ['video', 'text', 'mix', 'total']
    
    for idx, data in enumerate(datas):
        df = pd.read_csv(f'preprocessed_data/llm/predict_total/predict_{data}.csv', encoding='utf-8')
        df_d = pd.read_csv(f'data_kr/video/뉴스 영상 수집본.csv', encoding='utf-8')[['after', 'before']]
        df = pd.concat([df, df_d], axis=1)
        df[['upload_dt', 'after', 'before']] = df[['upload_dt', 'after', 'before']].apply(pd.to_datetime)
        
        ### LLM 예측에 실제 등락 라벨 추가 ###
        df["code"] = df["code"].astype(str).str.zfill(6)
        
        price_upload = [] # 업로드 당일 종가
        price_end = [] # 공시 당일 종가
        for row in tqdm(df.itertuples(), total=len(df)):
            df_price = pd.read_csv(f"data_kr/price/{row.code}.csv")
            df_price['날짜'] = pd.to_datetime(df_price['날짜'])
            ### 업로드 날짜 직전 종가
            price_upload.append(df_price.loc[df_price["날짜"] < row.upload_dt, "종가"].iloc[-1] if not pd.isna(row.upload_dt) else None)
            ### 데이터 업로드 주간 마지막 날 종가
            price_end.append(df_price.loc[df_price["날짜"] < row.before, "종가"].iloc[-1] if not pd.isna(row.before) else None)
        
        df["price_upload"] = price_upload
        df["price_end"] = price_end
    
        def check_change(row):
            rate = (row["price_end"] / row["price_upload"] - 1) * 100
            if rate > 0:
                return "상승"
            else:
                return "하락"
        df["label"] = df.apply(check_change, axis=1)

        ###############################################################
        ### 유효한 데이터만 남기기 ###
        valid_bool = ~(pd.isna(df['prediction']) | (df['prediction'] == '중립'))
        prediction = df[valid_bool]['prediction']
        label = df[valid_bool]['label']
        
        mapping = {'매우 긍정': '상승', '긍정': '상승', '중립': '횡보', '부정': '하락', '매우 부정': '하락'}
        prediction = [mapping[l] for l in prediction]
        

        accuracy = accuracy_score(label, prediction)
        precision = precision_score(label, prediction, pos_label='상승', zero_division=0)
        recall = recall_score(label, prediction, pos_label='상승', zero_division=0)
        f1 = f1_score(label, prediction, pos_label='상승', zero_division=0)

        score_dict = {'accuracy': accuracy, 'precision':precision, 'recall':recall, 'f1':f1}
        scores[data] = score_dict

    for data in datas:
        print(f'{data}\naccuracy: {scores[data]['accuracy']}\nprecision: {scores[data]['precision']}\nrecall: {scores[data]['recall']}\nf1: {scores[data]['f1']}\n')
get_pred_accuracy()

100%|███████████████████████████████████████████████████████████████████████| 5712/5712 [00:25<00:00, 226.41it/s]


video
accuracy: 0.5234042553191489
precision: 0.35118110236220473
recall: 0.27228327228327226
f1: 0.30674002751031637

text
accuracy: 0.5146666666666667
precision: 0.33724340175953077
recall: 0.11722731906218145
f1: 0.17397881996974282

mix
accuracy: 0.5721223895094706
precision: 0.3137254901960784
recall: 0.2028169014084507
f1: 0.24636441402908468

total
accuracy: 0.5596971461852067
precision: 0.3209718670076726
recall: 0.20373376623376624
f1: 0.24925521350546176

